In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import pickle
import numpy as np

from SlideRunner_dataAccess.database import Database
from lib.calculate_F1 import *
from lib.extractDetections import *
optimal_threshold, F1_values = {},{}

# Object detection (single shot) based on RetinaNet

## First: Clustering- and Object-Detection-augmented Expert Labelling (CODAEL)

We first optimize the threshold on the train+validation set, and then run inference on the test set.

In [2]:
databasefile = 'databases/MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite'

resfile = f'results/trainval_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'#
ident = f'CODAEL'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = f'results/test_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'#

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])


Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7464714606956605 Threshold= 0.6200000000000003
Calculating F1 for test set of 7 files
Overall: 
TP: 2586 FP: 1123 FN:  826 F1: 0.7263024856059542
Number of mitotic figures: 3412
Precision: 0.697 
Recall: 0.758


## Second: Object Detection-Augmented and Expert Labeled (ODAEL) data set variant

In this variant, an object-detection pipeline helped to find all mitotic figures and similar cells.

In [3]:
databasefile = 'databases/MITOS_WSI_CMC_ODAEL_TR.sqlite'

resfile = f'results/trainval_RetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'
ident = f'ODAEL'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = f'results/test_RetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7203881647797545 Threshold= 0.6700000000000004
Calculating F1 for test set of 7 files
Overall: 
TP: 2459 FP: 1118 FN:  1003 F1: 0.6986787896007955
Number of mitotic figures: 3462
Precision: 0.687 
Recall: 0.710


## Lastly: The initial manual labeling by two experts (MEL)

In [4]:
databasefile = 'databases/MITOS_WSI_CMC_MEL.sqlite'
resfile = f'results/trainval_RetinaNet-CMC-MEL-512sh-b1.pth-MEL-val-inference_results_boxes.p.bz2'#trainval_RetinaNet-CMC-MEL-512sh-b1.pth-MELshort-val-inference_results_boxes.p.bz2'
ident = f'MEL'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = f'results/test_RetinaNet-CMC-MEL-512sh-b1.pth-MEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7012792324605237 Threshold= 0.6200000000000003
Calculating F1 for test set of 7 files
Overall: 
TP: 2271 FP: 1075 FN:  1021 F1: 0.6842422416390479
Number of mitotic figures: 3292
Precision: 0.679 
Recall: 0.690


# Two-stage approach
In this approach, we use the results of the previous RetinaNet as a first stage, and a ResNet-18 classifier as secondary stage. It is being evaluated for all sets.


### First, again, the CODAEL data set variant.

In [5]:
databasefile = 'databases/MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite'

resfile = 'results/trainval_2ndstage_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'
ident = f'CODAEL_2nd'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = 'results/test_2ndstage_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.8407352871324751 Threshold= 0.7400000000000004
Calculating F1 for test set of 7 files
Overall: 
TP: 2659 FP: 794 FN:  753 F1: 0.774654042243263
Number of mitotic figures: 3412
Precision: 0.770 
Recall: 0.779


### Then, the ODAEL without any clustering help, detected by the dual stage setup:

In [6]:
databasefile = 'databases/MITOS_WSI_CMC_ODAEL_TR.sqlite'

resfile = 'results/trainval_2ndstage_CellClassifier_128px_ODAEL_b1.pthRetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'
ident = f'ODAEL_2nd'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = 'results/test_2ndstage_CellClassifier_128px_ODAEL_b1.pthRetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7937765475110963 Threshold= 0.8100000000000005
Calculating F1 for test set of 7 files
Overall: 
TP: 2604 FP: 1016 FN:  858 F1: 0.7353854843264614
Number of mitotic figures: 3462
Precision: 0.719 
Recall: 0.752


### And finally, the manual expert-labeled set with two stages

In [7]:

databasefile = 'databases/MITOS_WSI_CMC_MEL.sqlite'

resfile = 'results/trainval_RetinaNet-CMC-MEL-512sh-b1.pth-MEL-val-inference_results_boxes.p.bz2'
ident = f'MEL_2nd'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = 'results/trainval_2ndstage_RetinaNet-CMC-MEL-512sh-b1.pth-MELshort-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7012792324605237 Threshold= 0.6200000000000003
Calculating F1 for test set of 14 files
Overall: 
TP: 7204 FP: 4981 FN:  944 F1: 0.708601780357055
Number of mitotic figures: 8148
Precision: 0.591 
Recall: 0.884


### Print out table 2 in the manuscript

In [8]:
print('Single stage (RetinaNet, 512$\\times$512)'+f' & {F1_values["MEL"]:.3f} & {F1_values["ODAEL"]:.3f} & {F1_values["CODAEL"]:.3f} \\')
print(f'Dual stage (RetinaNet + ResNet-18) & {F1_values["MEL_2nd"]:.3f} & {F1_values["ODAEL_2nd"]:.3f} & {F1_values["CODAEL_2nd"]:.3f} \\')

Single stage (RetinaNet, 512$\times$512) & 0.684 & 0.699 & 0.726 \
Dual stage (RetinaNet + ResNet-18) & 0.709 & 0.735 & 0.775 \


# Transfer on human tissue

As described in the paper, we perform transfer tasks on human mammary carcinoma. For this, we detected mitoses on the publically available TUPAC16 set and its subsets. The
AMIDA13 set is the first 23 cases of the TUPAC16 set and was acquired with a different scanner (Aperio ScanScope), which makes the image acquisition parameters very similar to our image acquisition parameters and reduced the domain shift significantly.

Here, we test against the following conditions:

- AMIDA13: AMIDA13 complete set
- AMIDA13 test: AMIDA13 test set (last part of slides, used to compare against other approaches)
- TUPAC16 training set

We also relabeled the TUPAC16 set using almost the same methodology as in our paper and published this here:

- Bertram, C. A., Veta, M., Marzahl, C., Stathonikos, N., Maier, A., Klopfleisch, R., & Aubreville, M. (2020). Are pathologist-defined labels reproducible? Comparison of the TUPAC16 mitotic figure dataset with an alternative set of labels. In Interpretable and Annotation-Efficient Learning for Medical Image Computing (pp. 204-213). Springer, Cham. 
https://www.springerprofessional.de/are-pathologist-defined-labels-reproducible-comparison-of-the-tu/18445612

Find the complete dataset here:
https://github.com/DeepPathology/TUPAC16_AlternativeLabels

When refering to the alternative labels or AL, we refer to this alternative labels for the TUPAC16 set.




- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [9]:

databasefile = 'databases/MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite'

resfile = 'results/trainval_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'
ident = f'CODAEL_transfer'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amidatest_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,fa4959e484beec77543b.svs,72c93e042d0171a61012.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7464714606956605 Threshold= 0.6200000000000003
Calculating F1 for test set of 11 files
Overall: 
TP: 364 FP: 276.0 FN:  169 F1: 0.6206308610400681
Number of mitotic figures: 533
Precision: 0.569 
Recall: 0.683


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [10]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amidatest_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 11 files
Overall: 
TP: 437 FP: 202 FN:  171 F1: 0.7008821170809943
Number of mitotic figures: 608
Precision: 0.684 
Recall: 0.719


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)

In [11]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/tupactrain_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 73 files
Overall: 
TP: 1203.0 FP: 596.0 FN:  796.0 F1: 0.6334913112164297
Number of mitotic figures: 1999
Precision: 0.669 
Recall: 0.602


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)


In [12]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/tupactrain_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 73 files
Overall: 
TP: 954.0 FP: 850.0 FN:  598.0 F1: 0.5685339690107271
Number of mitotic figures: 1552
Precision: 0.529 
Recall: 0.615


## AMIDA13, complete set

- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [13]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amida_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 966 FP: 519 FN:  363 F1: 0.6865671641791045
Number of mitotic figures: 1329
Precision: 0.651 
Recall: 0.727


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [14]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amida_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 752 FP: 737.0 FN:  331 F1: 0.5847589424572317
Number of mitotic figures: 1083
Precision: 0.505 
Recall: 0.694


# RetinaNet only (single stage)

Now, for a comparison, only the first stage is evaluated.

- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [15]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amida_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 506 FP: 352.0 FN:  577 F1: 0.521380731581659
Number of mitotic figures: 1083
Precision: 0.590 
Recall: 0.467


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [16]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amida_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 612.0 FP: 247.0 FN:  717.0 F1: 0.5594149908592322
Number of mitotic figures: 1329
Precision: 0.712 
Recall: 0.460


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [17]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amidatest_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 11 files
Overall: 
TP: 237 FP: 119 FN:  296 F1: 0.5331833520809899
Number of mitotic figures: 533
Precision: 0.666 
Recall: 0.445


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [18]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amidatest_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 11 files
Overall: 
TP: 275.0 FP: 82 FN:  333.0 F1: 0.5699481865284974
Number of mitotic figures: 608
Precision: 0.770 
Recall: 0.452


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)


In [19]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/tupactrain_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 73 files
Overall: 
TP: 870.0 FP: 365.0 FN:  1129.0 F1: 0.5380333951762524
Number of mitotic figures: 1999
Precision: 0.704 
Recall: 0.435


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
